In [1]:
!pip install -qU langchain_community pypdf
!pip install langchain-groq
!pip install sentence_transformers
!pip install faiss-gpu
!pip install tiktoken
!pip install chromadb

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-cloud-bigquery 2.34.4 requires packaging<22.0dev,>=14.3, but you have packaging 24.1 which is incompatible.
jupyterlab 4.2.4 requires jupyter-lsp>=2.0.0, but you have jupyter-lsp 1.5.1 which is incompatible.
jupyterlab-lsp 5.1.0 requires jupyter-lsp>=2.0.0, but you have jupyter-lsp 1.5.1 which is incompatible.
libpysal 4.9.2 requires shapely>=2.0.1, but you have shapely 1.8.5.post1 which is incompatible.
momepy 0.7.2 requires shapely>=2, but you have shapely 1.8.5.post1 which is incompatible.
osmnx 1.9.4 requires shapely<2.1,>=2.0, but you have shapely 1.8.5.post1 which is incompatible.
pointpats 2.5.0 requires shapely>=2, but you have shapely 1.8.5.post1 which is incompatible.
spaghetti 1.7.6 requires shapely>=2.0.1, but you have shapely 1.8.5.post1 which is incompatible.
spopt 0.6.1 requires shapely>=2.0.

In [2]:
import os
os.environ['LANGCHAIN_TRACING_V2'] = 'true'
os.environ['LANGCHAIN_ENDPOINT'] = 'https://api.smith.langchain.com'
os.environ['LANGCHAIN_PROJECT'] = 'advanced-rag'
os.environ['LANGCHAIN_API_KEY'] = "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"
os.environ['GROQ_API_KEY'] = "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"

In [3]:
import bs4
from langchain import hub
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import FAISS
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain_groq import ChatGroq
from langchain_community.embeddings import HuggingFaceBgeEmbeddings
from langchain_community.document_loaders import PyPDFLoader
from langchain.prompts import ChatPromptTemplate
from langchain.storage import InMemoryByteStore
from langchain.retrievers.multi_vector import MultiVectorRetriever
from langchain_community.vectorstores import Chroma
from langchain_core.documents import Document
import uuid
import warnings
warnings.filterwarnings("ignore")

In [4]:
loader = PyPDFLoader('/kaggle/input/ecoli-and-bacterias/AVONCHEM.pdf')
docs =loader.load()

loader = PyPDFLoader('/kaggle/input/ecoli-and-bacterias/bacteria.pdf')
docs.extend(loader.load())

In [5]:
docs

[Document(metadata={'source': '/kaggle/input/ecoli-and-bacterias/AVONCHEM.pdf', 'page': 0}, page_content='AVONCHEM  \nMembrane Laury l Sulphate Broth ACM -1820 -O Pre -weighted size: 38.1g  \n \n1. What is M-Lauryl Sulfate Broth ? \nM-Lauryl Sulfate Broth is a type of broth used to count Escherichia coli  and other \ncoliform bacteria in water. It replaces Membrane Enriched Teepol Broth and is  \nrecommended by the ISO Committee according to ISO 9308 -1: 1990.  \n \n2. Composition:  \n \nIngredients     Grams/Litre  \n  Peptic digest of animal tissue   39.0 \nYeast extract                6.0 \nLactose     30.0 \nSodium lauryl sulfate    1.0  \nPhenol red     0.2  \nFinal pH 7.4 +/ - 0.2 at 25°C  \n \n3. Storage Instruct ions: \n\uf0b7 Store prepared media below 8°C.  \n\uf0b7 Protected from direct light.  \n\uf0b7 Store dehydrated powder, in a dry place, in tightly -sealed containers at 2 -25°C.  \n\uf0b7 The appearance of the broth is red colored  and clear . \n \n4. Preparation : \n\

In [6]:
chain = (
    {"doc": lambda x: x.page_content}
    | ChatPromptTemplate.from_template("Summarize the following document:\n\n{doc}")
    | ChatGroq()
    | StrOutputParser()
)

summaries = chain.batch(docs, {"max_concurrency": 1})

In [7]:
print(summaries)

['AVONCHEM Membrane Lauryl Sulfate Broth (ACM -1820 -O) is a broth used to enumerate Escherichia coli and other coliform bacteria in water, as an alternative to Membrane Enriched Teepol Broth and in line with ISO 9308 -1: 1990 recommendations. Its composition includes peptic digest of animal tissue (39.0 g/L), yeast extract (6.0 g/L), lactose (30.0 g/L), sodium lauryl sulfate (1.0 g/L), and phenol red (0.2 g/L), resulting in a final pH of 7.4 +/- 0.2 at 25°C. The powder should be stored in a dry place, while the prepared broth must be kept below 8°C, protected from light. Preparation involves suspending 76.2 g of the broth powder in 1000 ml of distilled water, boiling to dissolve, and sterilizing by steaming or autoclaving. Inoculation is carried out by filtering water samples through a sterile membrane and placing it on a sterile absorbent pad saturated with the broth.', 'The document outlines incubation procedures for testing water quality. For unchlorinated water, coliform organisms

In [8]:
model_name = "BAAI/bge-small-en"
model_kwargs = {"device": "cpu"}
encode_kwargs = {"normalize_embeddings": True}
hf_embeddings = HuggingFaceBgeEmbeddings(
    model_name=model_name, model_kwargs=model_kwargs, encode_kwargs=encode_kwargs)

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/90.8k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/52.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/684 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/366 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/711k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [9]:
# The vectorstore to use to index the child chunks
vectorstore = Chroma(collection_name="summaries",
                     embedding_function=hf_embeddings)

In [10]:
# The storage layer for the parent documents
store = InMemoryByteStore()
id_key = "doc_id"

In [11]:
# The retriever
retriever = MultiVectorRetriever(
    vectorstore=vectorstore,
    byte_store=store,
    id_key=id_key,
)
doc_ids = [str(uuid.uuid4()) for _ in docs]

In [12]:
# Docs linked to summaries
summary_docs = [
    Document(page_content=s, metadata={id_key: doc_ids[i]})
    for i, s in enumerate(summaries)
]

In [13]:
# Add
retriever.vectorstore.add_documents(summary_docs)
retriever.docstore.mset(list(zip(doc_ids, docs)))

In [14]:
doc_ids

['b016e1c8-6cab-4920-8232-018662349be5',
 '199ef161-c8b3-4864-bebb-3622d4bf63a6',
 'e68ec337-96cd-4536-84e9-6e4b702eb814']

In [15]:
retriever.docstore.mget(doc_ids)

[Document(metadata={'source': '/kaggle/input/ecoli-and-bacterias/AVONCHEM.pdf', 'page': 0}, page_content='AVONCHEM  \nMembrane Laury l Sulphate Broth ACM -1820 -O Pre -weighted size: 38.1g  \n \n1. What is M-Lauryl Sulfate Broth ? \nM-Lauryl Sulfate Broth is a type of broth used to count Escherichia coli  and other \ncoliform bacteria in water. It replaces Membrane Enriched Teepol Broth and is  \nrecommended by the ISO Committee according to ISO 9308 -1: 1990.  \n \n2. Composition:  \n \nIngredients     Grams/Litre  \n  Peptic digest of animal tissue   39.0 \nYeast extract                6.0 \nLactose     30.0 \nSodium lauryl sulfate    1.0  \nPhenol red     0.2  \nFinal pH 7.4 +/ - 0.2 at 25°C  \n \n3. Storage Instruct ions: \n\uf0b7 Store prepared media below 8°C.  \n\uf0b7 Protected from direct light.  \n\uf0b7 Store dehydrated powder, in a dry place, in tightly -sealed containers at 2 -25°C.  \n\uf0b7 The appearance of the broth is red colored  and clear . \n \n4. Preparation : \n\

In [16]:
query = "The heat for Escherichia coli "
sub_docs = vectorstore.similarity_search(query,k=1)
sub_docs[0]

Document(metadata={'doc_id': '199ef161-c8b3-4864-bebb-3622d4bf63a6'}, page_content='The document outlines incubation procedures for testing water quality. For unchlorinated water, coliform organisms should be incubated for 4 hours at 30°C then for an additional 14 hours at 35°C. Escherichia coli should be incubated for 4 hours at 30°C then for an additional 14 hours at 44°C. For chlorinated water, both coliform and Escherichia coli organisms benefit from an extended incubation of 6 hours at 25°C. After incubation, yellow colonies may form which need to be confirmed for the presence of the organisms.')

In [17]:
retrieved_docs = retriever.get_relevant_documents(query,n_results=1)
retrieved_docs[0].page_content[0:500]

' \n \n6. Incubation:  \n\uf0b7 Unchlorinated Waters : \n\uf0a7 Coliform organisms   \nFirst, incubate for 4 hours at 30°C.  \nThen, incubate for an additional 14 hours at 35°C.  \n \n\uf0a7 Escherichia coli:  \nFirst, incubate for 4 hours at 30°C.  \nThen, incubate for an additional 14 hours at 44°C.  \n \n\uf0b7 Chlorinated Waters:  \nThese organisms benefit from an extended incubation:  \nIncubate for 6 hours at 25°C.  \nAfter incubation, yellow colonies may form, which should be further confirmed to ensure \nthe presence of the'

# ColBERT

In [18]:
! pip install ragatouille

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.7/86.7 kB 4.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.7/41.7 kB 2.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 27.0/27.0 MB 39.1 MB/s eta 0:00:0000:01:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 171.5/171.5 kB 10.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.3/4.3 MB 16.6 MB/s eta 0:00:0000:0100:01m
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 51.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 47.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 55.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 288.3/288.3 kB 16.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.1/53.1 kB 3.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 187.4/187.4 kB 12.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 365.7/365.7 kB 22.1 MB/s et

In [19]:
from ragatouille import RAGPretrainedModel
RAG = RAGPretrainedModel.from_pretrained("colbert-ir/colbertv2.0")

artifact.metadata:   0%|          | 0.00/1.63k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/743 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/438M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/405 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

[Sep 06, 09:27:02] Loading segmented_maxsim_cpp extension (set COLBERT_LOAD_TORCH_EXTENSION_VERBOSE=True for more info)...


In [20]:
import requests

def get_wikipedia_page(title: str):
    """
    Retrieve the full text content of a Wikipedia page.

    :param title: str - Title of the Wikipedia page.
    :return: str - Full text content of the page as raw string.
    """
    # Wikipedia API endpoint
    URL = "https://en.wikipedia.org/w/api.php"

    # Parameters for the API request
    params = {
        "action": "query",
        "format": "json",
        "titles": title,
        "prop": "extracts",
        "explaintext": True,
    }

    # Custom User-Agent header to comply with Wikipedia's best practices
    headers = {"User-Agent": "RAGatouille_tutorial/0.0.1 (ben@clavie.eu)"}

    response = requests.get(URL, params=params, headers=headers)
    data = response.json()

    # Extracting page content
    page = next(iter(data["query"]["pages"].values()))
    return page["extract"] if "extract" in page else None

full_document = get_wikipedia_page("Document_retrieval")

In [21]:
full_document

'Document retrieval is defined as the matching of some stated user query against a set of free-text records. These records could be any type of mainly unstructured text, such as newspaper articles, real estate records or paragraphs in a manual. User queries can range from multi-sentence full descriptions of an information need to a few words.\nDocument retrieval is sometimes referred to as, or as a branch of, text retrieval. Text retrieval is a branch of information retrieval where the information is stored primarily in the form of text. Text databases became decentralized thanks to the personal computer. Text retrieval is a critical area of study today, since it is the fundamental basis of all internet search engines.\n\n\n== Description ==\nDocument retrieval systems find information to given criteria by matching text records (documents) against user queries, as opposed to expert systems that answer questions by inferring over a logical knowledge database. A document retrieval system

In [24]:
use_faiss=True
RAG.index(
    collection=[full_document],
    index_name="Doc-1",use_faiss=True,
    max_document_length=180,
    split_documents=True,
)

New index_name received! Updating current index_name (Doc-1) to Doc-1


[Sep 06, 09:39:28] #> Note: Output directory .ragatouille/colbert/indexes/Doc-1 already exists


[Sep 06, 09:39:28] #> Will delete 10 files already at .ragatouille/colbert/indexes/Doc-1 in 20 seconds...
[Sep 06, 09:39:49] [0] 		 #> Encoding 7 passages..


100%|██████████| 1/1 [00:01<00:00,  1.57s/it]

[Sep 06, 09:39:51] [0] 		 avg_doclen_est = 116.42857360839844 	 len(local_sample) = 7
[Sep 06, 09:39:51] [0] 		 Creating 256 partitions.
[Sep 06, 09:39:51] [0] 		 *Estimated* 815 embeddings.
[Sep 06, 09:39:51] [0] 		 #> Saving the indexing plan to .ragatouille/colbert/indexes/Doc-1/plan.json ..



WARNING clustering 775 points to 256 centroids: please provide at least 9984 training points


Clustering 775 points in 128D to 256 clusters, redo 1 times, 20 iterations
  Preprocessing in 0.00 s
[0.04, 0.033, 0.036, 0.033, 0.034, 0.031, 0.025, 0.032, 0.028, 0.027, 0.031, 0.024, 0.024, 0.031, 0.035, 0.034, 0.023, 0.027, 0.027, 0.031, 0.032, 0.031, 0.032, 0.031, 0.034, 0.031, 0.033, 0.032, 0.037, 0.033, 0.036, 0.036, 0.037, 0.041, 0.03, 0.029, 0.031, 0.034, 0.027, 0.037, 0.033, 0.031, 0.034, 0.032, 0.035, 0.024, 0.031, 0.033, 0.027, 0.031, 0.03, 0.039, 0.042, 0.031, 0.033, 0.048, 0.036, 0.04, 0.04, 0.032, 0.033, 0.045, 0.036, 0.04, 0.033, 0.043, 0.033, 0.033, 0.027, 0.031, 0.036, 0.024, 0.044, 0.031, 0.037, 0.027, 0.034, 0.035, 0.035, 0.036, 0.038, 0.039, 0.028, 0.039, 0.032, 0.029, 0.032, 0.033, 0.031, 0.036, 0.025, 0.027, 0.033, 0.036, 0.033, 0.022, 0.031, 0.032, 0.037, 0.026, 0.031, 0.035, 0.033, 0.03, 0.036, 0.028, 0.023, 0.021, 0.035, 0.027, 0.038, 0.033, 0.028, 0.035, 0.043, 0.037, 0.035, 0.034, 0.03, 0.03, 0.032, 0.035, 0.028, 0.038, 0.026, 0.034, 0.031, 0.03]


...


100%|██████████| 1/1 [00:01<00:00,  1.60s/it]
1it [00:01,  1.62s/it]
100%|██████████| 1/1 [00:00<00:00, 1502.80it/s]

[Sep 06, 09:39:52] #> Optimizing IVF to store map from centroids to list of pids..
[Sep 06, 09:39:52] #> Building the emb2pid mapping..
[Sep 06, 09:39:52] len(emb2pid) = 815



100%|██████████| 256/256 [00:00<00:00, 49816.36it/s]

[Sep 06, 09:39:52] #> Saved optimized IVF to .ragatouille/colbert/indexes/Doc-1/ivf.pid.pt
Done indexing!


'.ragatouille/colbert/indexes/Doc-1'

In [26]:
results = RAG.search(query="What is an example for form based indexing?", k=3)

In [27]:
results

[{'content': '== Variations ==\nThere are two main classes of indexing schemata for document retrieval systems: form based (or word based), and content based indexing. The document classification scheme (or indexing algorithm) in use determines the nature of the document retrieval system.\n\n\n=== Form based ===\nForm based document retrieval addresses the exact syntactic properties of a text, comparable to substring matching in string searches. The text is generally unstructured and not necessarily in a natural language, the system could for example be used to process large sets of chemical representations in molecular biology. A suffix tree algorithm is an example for form based indexing.',
  'score': 25.981781005859375,
  'rank': 1,
  'document_id': '791111a2-2420-45db-8254-163de09a81ab',
  'passage_id': 2},
 {'content': '== Example: PubMed ==\nThe PubMed form interface features the "related articles" search which works through a comparison of words from the documents\' title, abstr

In [28]:
retriever = RAG.as_langchain_retriever(k=3)
retriever.invoke("What is an example for form based indexing?")

[Document(page_content='== Variations ==\nThere are two main classes of indexing schemata for document retrieval systems: form based (or word based), and content based indexing. The document classification scheme (or indexing algorithm) in use determines the nature of the document retrieval system.\n\n\n=== Form based ===\nForm based document retrieval addresses the exact syntactic properties of a text, comparable to substring matching in string searches. The text is generally unstructured and not necessarily in a natural language, the system could for example be used to process large sets of chemical representations in molecular biology. A suffix tree algorithm is an example for form based indexing.'),
 Document(page_content='== Example: PubMed ==\nThe PubMed form interface features the "related articles" search which works through a comparison of words from the documents\' title, abstract, and MeSH terms using a word-weighted algorithm.\n\n\n== See also ==\nCompound term processing\n